### Yunjo Lee et Jean Marie Vilaire


In [35]:
using LinearAlgebra
using Pkg
Pkg.add("BenchmarkTools")
Pkg.activate("ProjetPartie3")

   Resolving package versions...
  No Changes to `~/Desktop/Desktop/Vilaire/POLY/Received_Courses/CoursPoly/MTH8211/Projet/Part3/ProjetPartie3/Project.toml`
  No Changes to `~/Desktop/Desktop/Vilaire/POLY/Received_Courses/CoursPoly/MTH8211/Projet/Part3/ProjetPartie3/Manifest.toml`
  Activating project at `~/Desktop/Desktop/Vilaire/POLY/Received_Courses/CoursPoly/MTH8211/Projet/Part3/ProjetPartie3`


In [36]:
function backsolve(R::UpperTriangular, b)
    x = similar(b)
    n = length(b)
    
    for i in n:-1:1
        if R[i, i] == 0
            throw(ArgumentError("Matrice singuliere a la ligne $i."))
        end
        x[i] = (b[i] - dot(R[i, i+1:n], x[i+1:n])) / R[i, i]
    end
    return x
end

#--------------------------------------------------------------------
function forwardsolve(L::LowerTriangular, b)
    x = similar(b)
    n = length(b)
    
    for i in 1:n
        if L[i, i] == 0
            throw(ArgumentError("Matrice singuliere a la ligne $i."))
        end
        x[i] = (b[i] - dot(L[i, 1:i-1], x[1:i-1])) / L[i, i]
    end
    return x
end

forwardsolve (generic function with 1 method)

In [55]:
function bk_partial_pivot(A)
    n = size(A, 1)
    α = (1 + sqrt(17)) / 8

    for k in 1:n
        if k < n
            λ = maximum(abs.(view(A, k+1:n, k)))
            if λ > 0
                if abs(A[k, k]) >= α * λ
                    return :un_par_un, k
                else
                    r = argmax(abs.(view(A, k+1:n, k)))[1] + k
                    σ = maximum(abs.(view(A, :, r)))
                    if abs(A[k, k]) * σ >= α^2 * λ^2
                        return :un_par_unPivot, k
                    elseif abs(A[r, r]) >= α * σ
                        return :un_par_unPivot, r
                    else
                        return :deux_par_deuxPivot, r
                    end
                end
            end
        else
            if abs(A[k, k]) > 0
                return :un_par_unPivot, k
            end
        end
    end
    return :NoPivot, n
end

bk_partial_pivot (generic function with 2 methods)

In [56]:
A = [1.0  1.0  1.0   1.0   1.0;
     1.0  3.0  3.0   3.0   3.0;
     1.0  3.0  6.0   7.0   7.0;
     1.0  3.0  7.0  12.0  12.0;
     1.0  3.0  7.0  12.0  17.0]
display(bk_partial_pivot(Hermitian(A)))


(:un_par_un, 1)

In [57]:
B = [0.0  1  0  0  0;
     0  0  0  0  0;
     0  0  0  1  2;
     0  0  1  0  0;
     0  0  2  0  0]
display(bk_partial_pivot(Hermitian(B)))

(:deux_par_deuxPivot, 2)

In [58]:
function bp_complete_pivot(A)
    n = size(A, 1)
    α = (1 + sqrt(17)) / 8
    pivots = []

    for k in 1:n-1
        ξ = maximum(abs.(A[i, j]) for i in 1:n, j in 1:n if i != j)
        η = maximum(abs.(A[i, i]) for i in 1:n)
        if ξ > 0 || η > 0
            λ = maximum(abs.(A[k, :]))
            if abs(A[k, k]) >= α * λ
                if η >= ξ
                    println("Utiliser A[$k, $k] comme pivot 1-par-1")
                    push!(pivots, (k, k, 1))
                else
                    println("Utiliser A[$k, $k] et A[$(k+1), $(k+1)] comme pivot 2-par-2")
                    push!(pivots, (k, k+1, 2))
                end
            end
        end
    end

    return pivots
end

bp_complete_pivot (generic function with 1 method)

In [59]:
A = [1.0  1.0  1.0   1.0   1.0;
     1.0  3.0  3.0   3.0   3.0;
     1.0  3.0  6.0   7.0   7.0;
     1.0  3.0  7.0  12.0  12.0;
     1.0  3.0  7.0  12.0  17.0]
pivots_A = bp_complete_pivot(Hermitian(A))

Utiliser A[1, 1] comme pivot 1-par-1
Utiliser A[2, 2] comme pivot 1-par-1
Utiliser A[3, 3] comme pivot 1-par-1
Utiliser A[4, 4] comme pivot 1-par-1


4-element Vector{Any}:
 (1, 1, 1)
 (2, 2, 1)
 (3, 3, 1)
 (4, 4, 1)

In [60]:
B = [0.0  1  0  0  0;
     0  0  0  0  0;
     0  0  0  1  2;
     0  0  1  0  0;
     0  0  2  0  0]
pivots_B = bp_complete_pivot(copy(B))

Utiliser A[2, 2] et A[3, 3] comme pivot 2-par-2


1-element Vector{Any}:
 (2, 3, 2)

## Bunch Kaufman

In [105]:
function bk_fact(A::Hermitian)
    n = size(A, 1)
    P = Matrix{Float64}(I, n, n)
    L = Matrix{Float64}(I, n, n)
    D = zeros(Float64, n, n)

    k = 1
    while k <= n
        pivot_type, pivot_index = bk_partial_pivot(A[k:n, k:n])

        if pivot_type == :un_par_un || pivot_type == :un_par_unPivot
            r = k + pivot_index - 1
            if r != k
                P[[k, r], :] = P[[r, k], :]
                A.data[[k, r], :] = A.data[[r, k], :]
                A.data[:, [k, r]] = A.data[:, [r, k]]
                L[[k, r], 1:k-1] = L[[r, k], 1:k-1]
            end

            D[k, k] = real(A[k, k])
            if k < n
                L[k+1:n, k] = A[k+1:n, k] / D[k, k]
                A.data[k+1:n, k+1:n] -= L[k+1:n, k] * conj.(A[k+1:n, k]')
            end
            k += 1
        elseif pivot_type == :deux_par_deuxPivot
            r = k + pivot_index - 1
            P[[k+1, r], :] = P[[r, k+1], :]
            A[[k+1, r], :] = [[r, k+1], :]
            A[:, [k+1, r]] = A[:, [r, k+1]]
            L[[k+1, r], 1:k-1] = L[[r, k+1], 1:k-1]
            if k+1 < n
                L[k+2:n, k:k+1] = A[k+2:n, k:k+1] / A[k:k+1, k:k+1]
                A[k+2:n, k+2:n] -= L[k+2:n, k:k+1] * A[k:k+1, k+2:n]
            end
            k += 2
        else 
            D[k, k] = A[k, k]
            break
        end
    end

    return P, L, D
end

bk_fact (generic function with 1 method)

In [106]:
A = [4.0 1.0 1 1;
     1 4 1 1;
     1 1 4 1;
     1 1 1 4]
A = Hermitian(A)
P, L, D = bk_fact(A)

display(L')
display(D)

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  0.25  0.25  0.25
 0.0  1.0   0.2   0.2
 0.0  0.0   1.0   0.166667
 0.0  0.0   0.0   1.0

4×4 Matrix{Float64}:
 4.0  0.0   0.0  0.0
 0.0  3.75  0.0  0.0
 0.0  0.0   3.6  0.0
 0.0  0.0   0.0  3.5

In [107]:
#K = bunchkaufman(A)

## Bunch Parlett

In [108]:
function bp_fact(A::Hermitian)
    n = size(A, 1)
    P = Matrix{Int64}(I, n, n)
    L = Matrix{Float64}(I, n, n)
    D = zeros(Float64, n, n)

    for k in 1:n
        # Find the pivot element
        max_abs_val = 0
        pivot_row = k
        for i in k:n
            if abs(A[i, k]) > max_abs_val
                max_abs_val = abs(A[i, k])
                pivot_row = i
            end
        end

        if pivot_row != k
            P[[k, pivot_row], :] = P[[pivot_row, k], :]
            A[[k, pivot_row], :] = A[[pivot_row, k], :]
            L[[k, pivot_row], :] = L[[pivot_row, k], :]
        end

        kk = A[k, k]
        if abs(kk) < 1e-10

            kp1 = k + 1
            D[k:kp1, k:kp1] = A[k:kp1, k:kp1]
            L[kp1, k] = A[kp1, k] / kk
            A[kp1, kp1] -= L[kp1, k] * A[kp1, k]
            k += 1
        else
            # Diagonal element is not close to zero, use a 1x1 block
            D[k, k] = kk
            for i in k+1:n
                L[i, k] = A[i, k] / kk
                A[i, i] -= L[i, k] * A[i, k]
            end
        end
    end

    return P, L, D
end
A = [4.0 1 1 1;
     1 4 1 1;
     1 1 4 1;
     1 1 1 4]

P, L, D = bp_fact(Hermitian(A))
# println("P:\n", P)
# println("L:\n", L)
# println("D:\n", D)
display(L')

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  0.25  0.25      0.25
 0.0  1.0   0.266667  0.266667
 0.0  0.0   1.0       0.287081
 0.0  0.0   0.0       1.0

## Rook Pivoting

In [120]:
function rook_pivoting(A::Matrix{Float64})
    n = size(A, 1)
    L = I(n)
    B = copy(A)
    P = I(n)
    Q = I(n)

    for j in 1:n-1
        i, k = findmax(abs.(B[j:n,j:n]), dims=(1,2))
        i += j-1
        k += j-1
        B[[j, i],:] = B[[i, j],:]
        L[[j, i],:] = L[[i, j],:]
        P[[j, i],:] = P[[i, j],:]
        B[:,[j, k]] = B[:,[k, j]]
        Q[:,[j, k]] = Q[:,[k, j]]
        for i in j+1:n
            L[i,j] = B[i,j] / B[j,j]
            B[i,:] -= L[i,j] * B[j,:]
        end
        if k > j
            d = B[j,j]
            c = B[k,j]
            b = B[j,k]
            a = B[k,k]
            det = a*d - b*c
            B[j:k,j:k] = [a b; c d] / det
        else
            B[j,j] = 1 / B[j,j]
        end
    end

    return L, B, P, Q
end

rook_pivoting (generic function with 1 method)

In [111]:
function solve_linear_myBP(A::Hermitian, b)#Solution avec notre implementaion de Bunch-Parlett
    x = similar(b)
    P, L, B = bp_fact(A)
    w = forwardsolve(LowerTriangular(L), P*b)
    z = B \ w
    y = backsolve(UpperTriangular(L'), z)
    x = P \ y
    return x
end

function solve_linear_myBK(A::Hermitian, b)#Solution avec notre implementaion de Bunch-Kaufman
    x = similar(b)
    P, L, B = bk_fact(A)
    w = forwardsolve(LowerTriangular(L), P*b)
    z = B \ w
    y = backsolve(UpperTriangular(L'), z)
    x = P \ y
    return x
end

function solve_linear_BP_LAPACK(A::Hermitian, b)#Solution avec LAPACK implementaion de Bunch-Parlett
    x = similar(b)
    #ldiv!(x,bunchparlett(A),b)
    #S = ldlt(A)
    ldiv!(x,ldlt(A),b)
    #x = S \ b
    return x
end
function solve_linear_BK_LAPACK(A::Hermitian, b)#Solution avec LAPACK de Bunch-Kaufman
    x = similar(b)
    ldiv!(x,bunchkaufman(A),b)
    return x
end

solve_linear_BK_LAPACK (generic function with 1 method)

In [122]:
function quadratic_optimization(Q, c, A, b)
    n = size(Q, 1)
    m = size(A, 1)
    K = [Q A'; A zeros(m, m)]
    K = Hermitian(K)
    rhs = [-c; b]
    sol_myBP = solve_linear_myBP(K, rhs)
    
    x_myBP = sol_myBP[1:n]
    λ_myBP = sol_myBP[n+1:end]

    return x_myBP, λ_myBP
end

n = 10 
m = 20 

Q = randn(n, n)
Q = Hermitian(Q'+Q)
c = randn(n)
A = randn(m, n)
b = randn(m)

x_opt, λ_opt = quadratic_optimization(Q, c, A, b)
println("Solution d'optimisation quadratique x:")
display(x_opt)
println("Multiplicateurs de Lagrange λ:")
display(λ_opt)


LoadError: ArgumentError: Cannot set a non-diagonal index in a Hermitian matrix

In [115]:
using BenchmarkTools

n = 5
m = 4
Q = randn(n, n)
Q = Q' + Q 
Q = Hermitian(Q)
b = randn(n);

In [116]:
@benchmark solve_linear_myBP($Q, $b)

LoadError: ArgumentError: Cannot set a non-diagonal index in a Hermitian matrix

In [117]:
@benchmark solve_linear_BK_LAPACK($Q, $b)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.385 μs …  1.434 ms  ┊ GC (min … max):  0.00% … 99.72%
 Time  (median):     1.598 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.055 μs ± 21.788 μs  ┊ GC (mean ± σ):  22.70% ±  2.22%

               ▃█▇▅                                           
  ▃▆▆▄▃▂▂▂▂▂▂▃▅████▆▃▂▂▂▁▂▂▂▁▂▂▂▁▁▁▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▂ ▃
  1.38 μs        Histogram: frequency by time        2.23 μs <

 Memory estimate: 3.06 KiB, allocs estimate: 5.

In [118]:
@benchmark solve_linear_myBK($Q, $b)

LoadError: DimensionMismatch: tried to assign 2-element array to 2×5 destination

In [119]:
@benchmark solve_linear_myBP($Q,$b)

LoadError: ArgumentError: Cannot set a non-diagonal index in a Hermitian matrix

In [25]:
function bk_pivot(A::Matrix{Float64}, α::Float64)
    n = size(A, 1)
    for k in 1:n
        if k < n
            λ = maximum(abs.(A[k+1:end, k]))
            if λ > 0
                if abs(A[k, k]) >= α * λ
                    return :un_par_un, k
                else
                    r = argmax(abs.(A[k+1:end, k])) + k
                    σ = maximum(abs.(A[:, r]))
                    if abs(A[k, k]) * σ >= α^2 * λ^2
                        return :un_par_unPivot, k
                    elseif abs(A[r, r]) >= α * σ
                        return :un_par_unPivot, r
                    else
                        return :deux_par_deuxPivot, r
                    end
                end
            end
        else
            if abs(A[k, k]) > 0
                return :un_par_unPivot, k
            end
        end
    end
    return :NoPivot, n
end


Lower triangular matrix L:
ComplexF64[1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.25 - 0.5im 1.0 + 0.0im 0.0 + 0.0im; 0.25 + 0.0im 0.0 + 0.0im 1.0 + 0.0im]

Block diagonal matrix D:
ComplexF64[4.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.75 + 0.0im 0.75 + 1.5im; 0.0 + 0.0im 0.75 - 1.5im 1.75 + 0.0im]

Permutation matrix P:
[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


In [26]:
A = [4.0 1 1 1;
     1 4 1 1;
     1 1 4 1;
     1 1 1 4]

L, D, P = LDL(Hermitian(A))
# println("P:\n", P)
# println("L:\n", L)
# println("D:\n", D)
display(L')

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [ ]:
function solve_linear_myBP(A::Hermitian, b)#Solution avec notre implementaion de Bunch-Parlett
    x = similar(b)
    P, L, B = bp_fact(A)
    w = forwardsolve(LowerTriangular(L), P*b)
    z = B \ w
    y = backsolve(UpperTriangular(L'), z)
    x = P \ y
    return x
end